# Creation d'un premier CNN simple

L'objectif de ce notebook est de créer un premier CNN simple qui servira de référence à la création d'un classifieur de comestibilité des champignons a partir d'une image.
Les inputs de ce notebook sont :
- le dataset d'images nettoyé et le fichier .csv correspondant au dataset d'images qui sera utilisé pour les parties train et test,
- le dataset de validation avec le fichier .csv associé.

Une visualisation durant l'apprentissage sera réalisée par le biais de GradCam.

In [ ]:
import os
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import time
import random
import seaborn as sns

import  keras
import tensorflow as tf # Utilisation de tensorflow v2.9.1
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Dropout, Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
# création des liens vers les dossiers et fichiers source
images_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\FFD_images_dataset'
train_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\train_FFDataframe_full_undersampling.csv'
test_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\test_FFDataframe_full_undersampling.csv'
validation_dataset = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\val_FFDataframe_full.csv'

# dossier ou sauver les résultats obtenus sur les modèles
save_models_results = r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\Dataset\Models_results'

In [ ]:
# création de quelques fonctions utiles

# affichage des metriques (accuracy, loss) d'entrainement d'un modèle
def plot_scores(model, title):
    '''
    Arg :
    model : model dont on souhaite afficher les metriques
    Return:
    plot des métriques Accuracy et loss sur les datasets train et test
    '''
    sns.set()
    plt.rcParams['figure.figsize'] = [14,4]

    # Créer la figure
    fig = plt.figure()
    
    plt.gcf().subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, wspace = 0.3, hspace = 0.3)
    # Créer les 4 graphiques
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)

    # Tracer les données sur les graphiques
    ax1.plot(model.history['accuracy'], label = "train")
    ax1.plot(model.history['val_accuracy'], label = "test")
    ax1.legend(loc = "lower right")
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')    

    ax2.plot(model.history['loss'], label = "train")
    ax2.plot(model.history['val_loss'], label = "test")
    ax2.legend(loc = "upper right")
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')  
    plt.title(title, loc = "left")
    plt.show()
    
# affichage de la matrice de confusion du dataset de validation
def show_confusion_matrix(model):
    '''
    Args :
    model : modele à utiliser pour fair eles predictions
   
    Return :
    plot de la matrice de confusion
    '''
    # réalisation des prédiction pour le modèle
    model_pred=model.predict(val_generator, steps=val_steps, verbose=1)
    y_pred = []
    for element in model_pred:
        pred = np.argmax(element)
        y_pred.append(pred)
    y_val = df_val.edible.to_list()
    confusion_mtx = confusion_matrix(y_val, y_pred)
    #
    plt.rcParams['font.size'] = 20
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mtx)
    disp.plot(cmap='Blues', values_format='d', xticks_rotation='horizontal', colorbar = False)
    plt.title(f'Confusion matrix for {model}')
    plt.ylabel('True label', fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.xlabel('Predicted label', fontsize = 20)
    plt.xticks(fontsize = 20)
    plt.grid(False)
    plt.show()
    


In [ ]:
# chargement des dataframes
df_train = pd.read_csv(train_dataset)
df_test = pd.read_csv(test_dataset)
df_val = pd.read_csv(validation_dataset)

# affichage de quelques infos sur ces dataframes + affichage d'une figure de répartition des catégories
display(df_train.head(), df_test.info(), df_val.info())

# génération des données du graph
inedible = []
edible = []

dataframes = [df_train, df_test, df_val]
for dataframe in dataframes:
    count_inedible = dataframe['edible'].value_counts()[0]
    inedible.append(count_inedible)
    count_edible = dataframe['edible'].value_counts()[1]
    edible.append(count_edible)   

data = ['df_train', 'df_test', 'df_val']
edibility = {'inedible': inedible, 'edible' : edible}

colonnes = ['df_train', 'df_test', 'df_val']
sex_counts = {
    'inedible': inedible,
    'edible': edible
}

width = 0.6
fig, ax = plt.subplots()
bottom = np.zeros(3)
for i, j in edibility.items():
    p = ax.bar(data, j, width, label=i, bottom=bottom)
    bottom += j
    ax.bar_label(p, label_type='center')
ax.set_title('Number of images by category')
ax.legend(title = 'categories')

plt.show()


## Mise en place d'un CNN d'architecture simple

### Création de générateurs de données 

In [ ]:
# Définition de quelques paramètres
batch_size = 64
SEED = 3
epochs = 15
W, H = 224, 224


In [ ]:
# Création d'un DataGenerator pour le dataset d'entrainement
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

df_train["edible"] = df_train["edible"].apply(str)

train_generator = train_datagen.flow_from_dataframe(df_train, images_dataset,
                                                    x_col="filename",
                                                    y_col="edible",
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=SEED)

In [ ]:
# Observation du fonctionnement du data generator sur quelques images du dataset d'entrainement
ex_df = df_train.sample(n=15).reset_index(drop=True)
ex_gen = train_datagen.flow_from_dataframe(ex_df,images_dataset,
                                           x_col="filename",
                                           y_col="edible",
                                           class_mode="categorical")

# affichage de quelquyes images issues du générateur
plt.figure(figsize=(15,15))
for i in range(0, 9):
    plt.subplot(5,3,i+1)
    for x, y in ex_gen:
        im = x[0]
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        break
plt.tight_layout()
plt.show()

In [ ]:
# Création d'un DataGenerator pour le dataset de test
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

df_test["edible"] = df_test["edible"].apply(str)

test_generator = test_datagen.flow_from_dataframe(df_test, images_dataset,
                                                  x_col="filename",
                                                  y_col="edible",
                                                  class_mode="categorical",
                                                  batch_size=batch_size)

## Création du reseau simple


In [ ]:
model_simple = Sequential()
# Convolution layer
model_simple.add(Conv2D(filters=16,
                 kernel_size=(3,3), 
                 padding='same',
                 use_bias=False,
                 input_shape=(224,224,3)))
# Pooling layer
model_simple.add(MaxPooling2D(pool_size=(4, 4),
                       strides=(4, 4),
                       padding='same'))
# Second convolution layer
model_simple.add(Conv2D(filters=32,
                 kernel_size=(3,3), 
                 padding='same',
                 use_bias=False))

model_simple.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))

model_simple.add(GlobalAveragePooling2D())
# Fully connected layers

model_simple.add(Dense(2, activation='softmax'))
model_simple.summary()

In [ ]:
# compilation du modèle
model_simple.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])


# entrainement du modèle
checkpointer_name = "model_simple.hdf5"
CSV_logger_name = "model_simple.csv"
checkpointer_model_simple= ModelCheckpoint(filepath=os.path.join(save_models_results, checkpointer_name),
                                            monitor='val_loss',
                                            save_best_only=True,
                                            mode='auto')
CSV_logger_model_simple = CSVLogger(filename = CSV_logger_name,
                                    separator=',',
                                    append = True)
callbacks_model_simple = [checkpointer_model_simple, CSV_logger_model_simple]
start_time = time.time()
history_model_simple = model.fit(train_generator,
                                                    epochs=epochs,
                                                    validation_data=test_generator,
                                                    validation_steps=len(df_test)//batch_size,
                                                    steps_per_epoch=len(df_train)//batch_size,
                                                    callbacks=callbacks_model_simple)
end_time = time.time()
print("Durée de l'entrainement :", end_time - start_time)

In [ ]:
plot_scores(history_model_simple, "scores du modèle CNN simple")

### Prédictions sur le dataset de validation et matrices de confusion 

In [ ]:
# Création d'un DataGenerator pour le dataset de validation
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

val_generator = val_datagen.flow_from_dataframe(df_val, images_dataset,
                                                  x_col="filename",
                                                  class_mode=None,
                                                  batch_size=1)
val_steps = len(df_val)
df_val["edible"] = df_val["edible"].apply(int)

In [ ]:
show_confusion_matrix(model)

In [ ]:
# affichage du classification report du modele simple
model_pred=model_simple.predict(val_generator, steps=val_steps, verbose=1)
y_pred = []
for element in model_pred:
    pred = np.argmax(element)
    y_pred.append(pred)
y_val = df_val.edible.to_list()
print(classification_report(y_val, y_pred))

## Conclusions
Ce premier modéle simple présente des résultats intéressants mais à tendance à classer de nombreuses images de champignons non-comestible comme étant comestibles ce qui est problématique... Dans de prochains notebook nous tenterons d'améliorer les résultats par utilisaitons de réseaux pré-entrainés.